In [ ]:
from tensorflow.keras.layers import Embedding, Dense,LSTM, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras import optimizers
from tensorflow import keras
from keras import models, layers
from sklearn.preprocessing import MinMaxScaler
import os

import numpy as np
import pandas as pd

In [ ]:
def change_time(time_str):
    time_str = str(time_str)
    return time_str[:4] + '/'+time_str[4:6] + '/' + time_str[6:]

def make_time(date_str):
    a = date_str.split(" ")
    t = a[0][:-1]+"/"+a[1][:-1]+"/"+a[2][:-1]
    return t

In [ ]:
high_yield = pd.read_csv("/content/drive/MyDrive/predict_price/high_yield.csv")
high_yield = high_yield.drop(high_yield[high_yield['BAMLH0A0HYM2'] == '.'].index)
high_yield['DATE'] = pd.to_datetime(high_yield['DATE'])

dollar_index = pd.read_csv("/content/drive/MyDrive/predict_price/dollar_index.csv")[['Date', 'Close']]
dollar_index['Date'] = pd.to_datetime(dollar_index['Date'])
krw = pd.read_csv("/content/drive/MyDrive/predict_price/krw.csv")[['Date','Close']]
krw['Date'] = pd.to_datetime(krw['Date'])
tnx = pd.read_csv("/content/drive/MyDrive/predict_price/^TNX.csv")[['Date', 'Close']]
tnx['Date'] = pd.to_datetime(tnx['Date'])
apply_data = pd.read_csv("/content/drive/MyDrive/predict_price/apply_data.csv")
apply_data['date'] = pd.to_datetime(apply_data['date'])

china = pd.read_csv("/content/drive/MyDrive/predict_price/sanhai.csv")[['날짜','종가']]
china['날짜'] = china['날짜'].apply(make_time)
china['날짜'] = pd.to_datetime(china['날짜'])
china['종가'] = china['종가'].apply(lambda x : x.replace(",",""))
china['종가'] = pd.to_numeric(china['종가'])
america = pd.read_csv("/content/drive/MyDrive/predict_price/america.csv")[['날짜','종가']]
america['날짜'] = america['날짜'].apply(make_time)
america['날짜'] = pd.to_datetime(america['날짜'])
america['종가'] = america['종가'].apply(lambda x : x.replace(",",""))
america['종가'] = pd.to_numeric(america['종가'])
wti = pd.read_csv('/content/drive/MyDrive/predict_price/wti.csv')[['날짜','종가']]
wti['날짜'] = wti['날짜'].apply(make_time)
wti['날짜'] = pd.to_datetime(wti['날짜'])
wti['종가'] = pd.to_numeric(wti['종가'])

In [ ]:
price = os.listdir("/content/drive/MyDrive/창의학기제/real_final_data")

for src in price[1:]:
    data = pd.read_csv(f'/content/drive/MyDrive/창의학기제/real_final_data/{src}')
    data = data.set_index('Date')
    scaler = MinMaxScaler()
    data_processing = scaler.fit_transform(data)
    data_processing = pd.DataFrame(data_processing)

    X_l = []
    y_l = []
    N = len(data_processing)
    D = 5
    for i in range(N-D-1):
        X_l.append(data_processing.iloc[i:i+D])
        y_l.append(data_processing[0].iloc[i+D])
    X = np.array(X_l)
    y = np.array(y_l)
    model = Sequential()
    model.add(LSTM(20,input_shape=X.shape[1:]))
    model.add(Dense(10))
    model.add(Dense(1))
    model.compile('adam','mse')
    history = model.fit(X,y, batch_size=64, epochs=100, validation_split=0.3,verbose=1)
    print(f"{price_src} - va_loss : ", history.history['val_loss'][-1])
    model.save(f'/content/drive/MyDrive/real_final/{src}.h5')

In [ ]:
for price_src in price:
    print(price_src)
    data = pd.read_csv(f"/content/drive/MyDrive/price_data/{price_src}")[['Date','Close']]
    # data['Date'] = data['Date'].apply(change_time)
    data['Date'] = pd.to_datetime(data['Date'])
    data = pd.merge(data, high_yield, left_on='Date', right_on='DATE',).drop('DATE',axis=1)
    data = pd.merge(data,dollar_index, on ='Date')
    data = pd.merge(data, krw ,on ='Date')
    data = pd.merge(data, tnx, on="Date")
    data = pd.merge(data, apply_data, left_on='Date', right_on='date').drop('date',axis=1)
    data = pd.merge(data, china, left_on='Date', right_on='날짜').drop('날짜',axis=1)
    data = pd.merge(data, america, left_on='Date', right_on='날짜').drop('날짜',axis=1)
    data = pd.merge(data, wti, left_on='Date', right_on='날짜').drop('날짜',axis=1)

    data.columns = ['Date','price','high_yield','dollar_index','krw','tnx','organization','foreigner','china','america','wti']
    data = data[1:]

    if len(data) <= 365 * 3:
        continue

    control_columns_null = data.columns[2:]

    for c in control_columns_null:
        for i in data[data[c].isnull()].index:
            data.loc[i,c] = data.loc[i-1,c]
    data.to_csv(f"{price_src}",index= None)
    data = data.set_index('Date')
    tmp =price_src.split("_")[0]
    data.to_csv(f"/content/drive/MyDrive/price_data/{tmp}.csv")




In [ ]:
scaler = MinMaxScaler()
data_processing = scaler.fit_transform(np.array(data))
data_processing = pd.DataFrame(data_processing)

X_l = []
y_l = []
N = len(data_processing)
D = 5
for i in range(N-D-1):
    X_l.append(data_processing.iloc[i:i+D])
    y_l.append(data_processing[0].iloc[i+D])
X = np.array(X_l)
y = np.array(y_l)

model = Sequential()
model.add(LSTM(20,input_shape=X.shape[1:]))
model.add(Dense(10,activation='relu'))
model.add(Dense(1))
model.compile('adam','mse')
history = model.fit(X,y, batch_size=64, epochs=200, validation_split=0.3,verbose=0)
print(f"{price_src} - va_loss : ", history.history['val_loss'][-1])
model.save(f'/content/drive/MyDrive/model_file/{price_src}.h5')

In [ ]:
price = os.listdir("/content/drive/MyDrive/predict_price/price")

for price_src in price:
    data = pd.read_csv(f"/content/drive/MyDrive/price_data/{price_src}")

    scaler = MinMaxScaler()
    data_processing = scaler.fit_transform(np.array(data))
    data_processing = pd.DataFrame(data_processing)

    X_l = []
    y_l = []
    N = len(data_processing)
    D = 5
    for i in range(N-D-1):
        X_l.append(data_processing.iloc[i:i+D])
        y_l.append(data_processing[0].iloc[i+D])
    X = np.array(X_l)
    y = np.array(y_l)

    model = Sequential()
    model.add(LSTM(20,input_shape=X.shape[1:]))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(1))
    model.compile('adam','mse')
    history = model.fit(X,y, batch_size=64, epochs=200, validation_split=0.3,verbose=0)
    print(f"{price_src} - va_loss : ", history.history['val_loss'][-1])
    model.save(f'/content/drive/MyDrive/model_file/{price_src}.h5')

In [ ]:
logo_time = os.listdir("/content/drive/MyDrive/predict_price/price")

In [ ]:
logo_name = [*map(lambda x : x[:-4],logo_time)]

In [ ]:
logo_name = [*map(lambda x : x.split("_"), logo_name)]

In [ ]:
stock_list = {}
for code, name in logo_name:
    stock_list[code] = name

In [ ]:
models_h5 = os.listdir('/content/drive/MyDrive/model_h5')
price_data = os.listdir('/content/drive/MyDrive/price_data')
arr = []

for mod in models_h5:
    data = pd.read_csv(f"/content/drive/MyDrive/price_dataset/{mod.split('.')[0]}.csv")
    scaler = MinMaxScaler()
    data = data[data.columns[1:]]
    data_processing = scaler.fit_transform(np.array(data))
    data_processing = pd.DataFrame(data_processing)
    new_model = models.load_model(f'/content/drive/MyDrive/model_h5/{mod}')

    a= new_model.predict(np.array(data_processing[-6:-1]).reshape(-1,5,10))
    va = (a - data_processing.iloc[-2][0])/ data_processing.iloc[-2][0] * 100

    if -5 <= va <= 5:
        print(stock_list[mod.split(".")[0]]," : ",round(float(va),2))
        arr.append([stock_list[mod.split(".")[0]],round(float(va),2)])
    elif -50<= va <= -5 or 5 <= va <=50:
        print(stock_list[mod.split(".")[0]]," : ",round(float(va)/10,2) )
        arr.append([stock_list[mod.split(".")[0]],round(float(va)/10,2)])
    else:
        print(stock_list[mod.split(".")[0]]," : ",round(float(va)/100,2))
        arr.append([stock_list[mod.split(".")[0]],round(float(va)/100,2)])


LG  :  0.97
쌍용C&E  :  -1.23
대한항공  :  -1.29
아모레G  :  4.05
삼성화재  :  2.42
현대건설  :  -1.06
SK하이닉스  :  2.19
기아  :  0.03
CJ대한통운  :  3.66
유한양행  :  -1.69
현대모비스  :  -1.43
금호석유  :  -2.06
SKC  :  -3.19
HMM  :  -1.11
롯데케미칼  :  1.67
LG이노텍  :  -1.07
S-Oil  :  1.15
현대미포조선  :  0.79
삼성중공업  :  -0.54
고려아연  :  0.74
한화솔루션  :  -0.72
한국조선해양  :  -1.38
삼성전기  :  -0.59
한미사이언스  :  4.96
호텔신라  :  -0.64
메리츠증권  :  -0.91
GS리테일  :  -2.5
미래에셋증권  :  -0.66
삼성SDI  :  -3.9
녹십자  :  -0.95
NH투자증권  :  -0.67
GS건설  :  -2.23
삼성전자  :  -5.52
DB손해보험  :  0.97
POSCO  :  -3.11
현대차  :  -1.11
롯데지주  :  0.69
현대제철  :  -1.14
포스코케미칼  :  0.26
한국전력  :  -1.91
SK텔레콤  :  -4.18
삼성증권  :  -0.9
삼성에스디에스  :  -2.28
삼성엔지니어링  :  -0.55
LG유플러스  :  -0.77
삼성카드  :  -2.63
KT  :  -2.9
팬오션  :  -0.25
일진ᄆ

In [ ]:
arr

[['LG', 0.97],
 ['쌍용C&E', -1.23],
 ['대한항공', -1.29],
 ['아모레G', 4.05],
 ['삼성화재', 2.42],
 ['현대건설', -1.06],
 ['SK하이닉스', 2.19],
 ['기아', 0.03],
 ['CJ대한통운', 3.66],
 ['유한양행', -1.69],
 ['현대모비스', -1.43],
 ['금호석유', -2.06],
 ['SKC', -3.19],
 ['HMM', -1.11],
 ['롯데케미칼', 1.67],
 ['LG이노텍', -1.07],
 ['S-Oil', 1.15],
 ['현대미포조선', 0.79],
 ['삼성중공업', -0.54],
 ['고려아연', 0.74],
 ['한화솔루션', -0.72],
 ['한국조선해양', -1.38],
 ['삼성전기', -0.59],
 ['한미사이언스', 4.96],
 ['호텔신라', -0.64],
 ['메리츠증권', -0.91],
 ['GS리테일', -2.5],
 ['미래에셋증권', -0.66],
 ['삼성SDI', -3.9],
 ['녹십자', -0.95],
 ['NH투자증권', -0.67],
 ['GS건설', -2.23],
 ['삼성전자', -5.52],
 ['DB손해보험', 0.97],
 ['POSCO', -3.11],
 ['현대차', -1.11],
 ['롯데지주', 0.69],
 ['현대제철', -1.14],
 ['포스코케미칼', 0.26],
 ['한국전력', -1.91],
 ['SK텔레콤', -4.18],
 ['삼성증권', -0.9],
 ['삼성에